# ChatFlow Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json, copy

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_threads, run_dispatch_worker_thread
from aiflows import flow_verse

dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "main"},
    {"url": "aiflows/HumanStandardInputFlowModule", "revision": "main"},
    {"url": "aiflows/ChatInteractiveFlowModule", "revision": "main"}
]
flow_verse.sync_dependencies(dependencies)

[2024-03-17 07:22:40,011][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules...
[2024-03-17 07:22:40,140][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-17 07:22:40,249][aiflows.flow_verse.loading:608][INFO] - aiflows/HumanStandardInputFlowModule:coflows already synced, skip
[2024-03-17 07:22:40,611][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatInteractiveFlowModule:coflows already synced, skip
[2024-03-17 07:22:40,613][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules/aiflows/ChatInteractiveFlowModule',
 '/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules/aiflows/ChatFlowModule',
 '/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules/aiflows/HumanStandardInputFlowModule']

# Connect to CoLink Server

In [3]:
# This example only works with a server outside of the notebook because we will need to also start the human UI in a separate terminal
USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
user_jwt = open(USER_JWT_PATH).readline().strip()
cl = CoLink("http://127.0.0.1:2021", user_jwt)

# Start a few default workers

In [4]:
run_dispatch_worker_thread(cl)
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch


ERROR:root:CreateEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name already exists: 0346f4875c8cb3dd5660afc51b69e60c8eddaefa7be820a09b840eca11e005fa04::_lock:_internal:protocols:coflows_dispatch:local:operator_mq
ERROR:root:CreateEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name already exists: 0346f4875c8cb3dd5660afc51b69e60c8eddaefa7be820a09b840eca11e005fa04::_lock:_internal:protocols:coflows_dispatch:local:operator_mq



~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: 3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74
owner_id: local
message_paths: ['push_tasks:1cba240f-fe6f-4c54-a60e-a5c310625bea:msg']
parallel_dispatch: False

Input message source: unknown

~~~ Dispatch task ~~~
flow_endpoint: Assistant
flow_id: 5b117a6f-151d-4dcf-9435-9b092bd83d49
owner_id: local
message_paths: ['push_tasks:9757423c-6df5-478f-9346-53eaaeaaec82:msg']
parallel_dispatch: False

Input message source: Assistant
CHATFLOW YO, input: {'id': 0, 'query': 'I want to ask you a few questions', 'answer': None, 'end_of_interaction': False}

~~~ Dispatch task ~~~
flow_endpoint: Assistant
flow_id: b5d7a464-cc76-4d9d-8195-058f81d2fe96
owner_id: local
message_paths: ['push_tasks:f63737a2-15fc-48ad-9418-7611f5afbf13:msg']
parallel_dispatch: False

Input message source: Assistant
CHATFLOW YO, input: {'id': 0, 'query': 'I want to ask you a few questions. Please ONLY respond in Croatian.', 'answer': None, 'end_of_interaction': False}
CHA

In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: 67f237f1-7a3c-4dee-8904-17ad3240468e
owner_id: local
message_paths: ['push_tasks:7dd17c06-0daf-475a-85db-9b0f971feddd:msg']
parallel_dispatch: False

Input message source: unknown

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: 3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74
owner_id: local
message_paths: ['push_tasks:77d56759-fd83-4284-afd2-5d5af69fb515:msg']
parallel_dispatch: False

Input message source: Assistant

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: 3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74
owner_id: local
message_paths: ['push_tasks:04c6357b-aae3-422c-8a36-dfa1a71e5124:msg']
parallel_dispatch: False

Input message source: User

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: 67f237f1-7a3c-4dee-8904-17ad3240468e
owner_id: local
message_paths: ['push_tasks:555b8ed3-e75a-435f-bd16-8565f630a5c4:msg']
parallel

# Serve

User should start dispatch_worker with dispatch_point="human_dispatch" in a separate terminal (serves as UI for user). Plz ensure that you pass the correct flow_modules_base_path when starting worker (give path to flow_modules folder that contains HumanStandardInputFlowModule being served here.

In [6]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.HumanStandardInputFlowModule.HumanStandardInputFlow",
    flow_endpoint="Human",
    singleton=True,
    dispatch_point="human_dispatch",
)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Assistant",
)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatInteractiveFlowModule.ChatHumanFlowModule",
    flow_endpoint="InteractiveChat",
)

Started serving flow_modules.aiflows.HumanStandardInputFlowModule.HumanStandardInputFlow at flows:Human.
dispatch_point: human_dispatch
parallel_dispatch: False
singleton: True
Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Assistant.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False
Started serving flow_modules.aiflows.ChatInteractiveFlowModule.ChatHumanFlowModule at flows:InteractiveChat.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

## First instance

In [7]:
root_dir = "."
cfg_path = os.path.join(root_dir, "flow_modules/aiflows/ChatInteractiveFlowModule/demo.yaml")
cfg = read_yaml_file(cfg_path)

In [8]:
interactive_chat = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="InteractiveChat",
    config_overrides=cfg,
)

input_message = FlowMessage(
    data={"id": 0, "query": "I want to ask you a few questions"},
)
interactive_chat.send_message(input_message)

[2024-03-17 07:22:41,270][aiflows.flow_verse.loading:775][INFO] - [flow_modules.aiflows.ChatInteractiveFlowModule] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules...
[2024-03-17 07:22:41,367][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-17 07:22:41,464][aiflows.flow_verse.loading:608][INFO] - aiflows/HumanStandardInputFlowModule:coflows already synced, skip
[2024-03-17 07:22:41,465][aiflows.flow_verse.loading:825][INFO] - [flow_modules.aiflows.ChatInteractiveFlowModule] finished syncing


Mounted 5b117a6f-151d-4dcf-9435-9b092bd83d49 at flows:Assistant:mounts:local:5b117a6f-151d-4dcf-9435-9b092bd83d49
Mounted edf261a1-5f7e-41df-9c2c-2760466e5b79 at flows:Human:mounts:local:edf261a1-5f7e-41df-9c2c-2760466e5b79
Mounted 3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74 at flows:InteractiveChat:mounts:local:3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74


## Second instance

In [9]:
interactive_chat_CRO = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="InteractiveChat",
    config_overrides=cfg,
)

input_message = FlowMessage(
    data={"id": 0, "query": "I want to ask you a few questions. Please ONLY respond in Croatian."},
)
interactive_chat_CRO.send_message(input_message)

Mounted b5d7a464-cc76-4d9d-8195-058f81d2fe96 at flows:Assistant:mounts:local:b5d7a464-cc76-4d9d-8195-058f81d2fe96
Fetched singleton edf261a1-5f7e-41df-9c2c-2760466e5b79
Mounted 67f237f1-7a3c-4dee-8904-17ad3240468e at flows:InteractiveChat:mounts:local:67f237f1-7a3c-4dee-8904-17ad3240468e


## Highlights

We have two instances of InteractiveChatFlow - they share the same singleton Human subflow and have separate Assistant subflows.

In [10]:
def status():
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)
status()

 Human/
   init
   default_dispatch_point
   flow_class_name
   singleton
   parallel_dispatch
   mounts/
     local/
       edf261a1-5f7e-41df-9c2c-2760466e5b79/
         init
         config_overrides
 Assistant/
   parallel_dispatch
   singleton
   flow_class_name
   default_dispatch_point
   mounts/
     local/
       b5d7a464-cc76-4d9d-8195-058f81d2fe96/
         init
         config_overrides
       5b117a6f-151d-4dcf-9435-9b092bd83d49/
         init
         config_overrides
         state
   init
 InteractiveChat/
   init
   flow_class_name
   default_dispatch_point
   parallel_dispatch
   mounts/
     local/
       3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74/
         config_overrides
         state
         init
       67f237f1-7a3c-4dee-8904-17ad3240468e/
         state
         config_overrides
         init
   singleton

All flow instances:
 67f237f1-7a3c-4dee-8904-17ad3240468e
 3a3ee7a4-4b00-4f2a-b80d-3bd7dbf52c74
 5b117a6f-151d-4dcf-9435-9b092bd83d49
 edf261a1-5f7e-41df-9c2c-27